In [1]:
# 필요한 라이브러리  import
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import pandas as pd

In [2]:
# CSV 파일에서 데이터셋을 로드
file_path = '../../data/csv/1.01.house.price.csv'
data = pd.read_csv(file_path)

In [3]:
###### 데이터 전처리
# 데이터셋을 PyTorch 텐서로 변환
X = torch.tensor(data[['평수', '연식']].values, dtype=torch.float32)
y = torch.tensor(data['가격 '].values, dtype=torch.float32).view(-1, 1)

# 스케일링
X_norm = (X - X.mean(dim=0)) / X.std(dim=0)
    # 표준화(Z-Score 정규화): 데이터를 평균이 0이고 표준편차가 1인 분포로 변환. (각 데이터에서 평균을 빼고, 그 결과를 표준편차로 나눔)
    # 해석할때는 역변환 해야함

    
# scaler_X = StandardScaler()
# scaler_y = StandardScaler()
# X_scaled = scaler_X.fit_transform(X)
# y_scaled = scaler_y.fit_transform(y)


In [4]:

# 선형 회귀 모델 정의
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(2, 1)  # 입력 차원은 2 ('평수'와 '연식'), 출력 차원은 1 ('가격')

    def forward(self, x):
        return self.linear(x)
        # 입력 데이터 x에 대해 self.linear 선형 레이어를 적용하고, 결과를 반환
        # 선형 레이어는 내부적으로 학습된 가중치와 편향을 사용하여 계산을 수행

# 모델 인스턴스를 생성합니다.
model = LinearRegressionModel()


In [5]:

# 손실 함수로 평균 제곱 오차(MSE)를 사용
criterion = nn.MSELoss()

# 최적화 알고리즘으로 SGD를 사용하며, 학습률은 0.001로 설정
optimizer = optim.SGD(model.parameters(), lr=0.0001)


In [6]:

######### 모델 훈련
num_epochs = 1000
for epoch in range(num_epochs):
    # Variable로 데이터를 감싸 계산 그래프
    inputs = Variable(X)
    targets = Variable(y)

    # 순전파 단계: 예측값을 계산
    outputs = model(inputs)
    # 손실을 계산
    loss = criterion(outputs, targets)

    # 역전파 단계: 기울기를 초기화하고, 손실에 대한 기울기를 계산한 다음, 최적화 단계를 수행
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 일정한 에포크마다 현재의 손실을 출력
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 3.5951
Epoch [200/1000], Loss: 1.3367
Epoch [300/1000], Loss: 0.6752
Epoch [400/1000], Loss: 0.4813
Epoch [500/1000], Loss: 0.4244
Epoch [600/1000], Loss: 0.4075
Epoch [700/1000], Loss: 0.4024
Epoch [800/1000], Loss: 0.4008
Epoch [900/1000], Loss: 0.4001
Epoch [1000/1000], Loss: 0.3998


In [7]:

########### 모델 평가: 학습된 모델을 사용하여 첫 번째 데이터 포인트의 가격 예측
model.eval()
with torch.no_grad():
    predicted = model(Variable(X[0]))
    # 정규화된 예측값을 원래 스케일로 복원
    predicted_price = predicted.item() * y.std() + y.mean()
    print(f'첫 번째 아파트의 예측 가격: {predicted_price:.4f}, 실제 가격: {y[0].item():.4f}')

# 훈련된 모델의 매개변수를 출력
print(f'모델의 가중치: {model.linear.weight}')
print(f'모델의 편향: {model.linear.bias}')


첫 번째 아파트의 예측 가격: 8.0791, 실제 가격: 5.0000
모델의 가중치: Parameter containing:
tensor([[ 0.1394, -0.0916]], requires_grad=True)
모델의 편향: Parameter containing:
tensor([-0.4782], requires_grad=True)


In [8]:
# 전체 테스트 데이터셋에 대한 예측 수행
model.eval()  # 모델을 평가 모드로 설정
with torch.no_grad():
    predictions = model(X)

# predictions

In [9]:
# 성능 지표 계산

# 스케일이 되어있기때문에 이를 풀어야한다.
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y.numpy(), predictions.numpy())
rmse = np.sqrt(mse)
mae = mean_absolute_error(y.numpy(), predictions.numpy())

print(f'MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}')

MSE: 0.3998, RMSE: 0.6323, MAE: 0.4701
